## Install dependencies

In [ ]:
import sklearn
import xgboost
import lightgbm
import numpy as np
print(f'{sklearn.__version__} {xgboost.__version__} {lightgbm.__version__} {np.__version__}')

In [ ]:
! pip install -U matplotlib pandas scikit-learn xgboost lightgbm umap-learn xopen orjson tqdm

## Prepare the notebook

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Calculate number of CPUs

In [ ]:
import multiprocessing

In [ ]:
workers = multiprocessing.cpu_count()
workers

# Import the libraries

In [ ]:
from pathlib import Path
from typing import Union, Any
import gc

In [ ]:
import joblib
from tqdm import tqdm

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split

## File locations for training and validation / test data

In [ ]:
path = Path('data')
DATA_DIR = path / 'www'
DATA_DIR.mkdir(exist_ok=True, parents=True)

In [ ]:
NODES_PATH = DATA_DIR / 'nodes.joblib'

In [ ]:
! ls {DATA_DIR}

# Download the data from the bucket

In [ ]:
! gsutil -m cp gs://www21/nodes.joblib {NODES_PATH}

In [ ]:
! ls {DATA_DIR}

## Load training data

Load training data `X_train` and `y_train` and validation / test data `X_test` and `y_test` with `80% / 20%` ratio

In [ ]:
X_train, X_test, y_train, y_test = joblib.load(NODES_PATH)

## Analyze data

In [ ]:
from sklearn.manifold import TSNE
from umap import UMAP

In [ ]:
def show_distribution(frequencies:np.ndarray):
    """Plot distribution of labels"""
    labels = ['tag', 'ad']
    fig, ax = plt.subplots()
    plt.figure(figsize=(40, 40))
    im = ax.imshow(frequencies, cmap='viridis', interpolation='nearest')
    ax.set_xticks(np.arange(len(labels)))
    ax.set_xticklabels([])
    ax.set_yticks(np.arange(len(labels)))
    ax.set_yticklabels([])
    thresh = (frequencies.max() + frequencies.min()) / 2.0
    cmap_min, cmap_max = im.cmap(0), im.cmap(256)
    for i in range(2):
        for j in range(2):
            color = cmap_max if frequencies[i, j] < thresh else cmap_min
            freq_val = labels[i] if j == 0 else frequencies[i, j]
            text = ax.text(j, i, freq_val, ha='center', va='center', color=color)
    ax.set_title('Distribution of tag / ad labels')
    fig.tight_layout()
    plt.show()

#### Estimate data and label distribution

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
unique_tr, counts_tr = np.unique(y_train, return_counts=True)
frequencies_tr = np.asarray((unique_tr, counts_tr)).T
frequencies_tr
print(f' nodes: {y_train.shape[0]}, ads: {counts_tr[1]}, ads%: {(y_train.shape[0] / counts_tr[1]) * 100:.2f}%')
show_distribution(frequencies_tr)

In [ ]:
unique_ts, counts_ts = np.unique(y_test, return_counts=True)
frequencies_ts = np.asarray((unique_ts, counts_ts)).T
print(f' nodes: {y_test.shape[0]}, ads: {counts_ts[1]}, ads%: {(y_test.shape[0] / counts_ts[1]) * 100:.2f}%')
show_distribution(frequencies_ts)

In [ ]:
node_count = y_train.shape[0] + y_test.shape[0]
ad_count = counts_tr[1] + counts_ts[1]
print(f' nodes: {node_count}, ads: {ad_count}, ads%: {(ad_count / node_count) * 100:.2f}%')

#### Project nodes on `2D` plane with `UMAP` and `T-SNE`

Project node features on `2D` Euclidean plane preserving metrics (distances) between nodes using `UMAP` and `T-SNE` dimensionality reduction algorithms

In [ ]:
def project_nodes(X:np.ndarray, y:np.ndarray, num_classes:int=2, 
                  proj_type:type=UMAP, **kwargs) -> np.ndarray:
    """Project nodes on 2D plane"""
    return proj_type(n_components=num_classes, **kwargs).fit_transform(X, y=y)

#### Plot embeddings on `2D` plane 

In [ ]:
def plot_nodes(z:np.ndarray, y:np.ndarray, colors:list,
               proj_type:type=UMAP, num_classes:int=2, **kwargs):
    plt.figure(figsize=(8, 8))
    for i in range(num_classes):
        plt.scatter(z[y == i, 0], z[y == i, 1], s=20, color=colors[i])
    plt.axis('off')
    plt.title(f'Node features via {proj_type.__name__} using Labels');
    plt.show()

In [ ]:
colors = ['#ffc0cb', '#bada55']

In [ ]:
tail = 500

In [ ]:
embs_umap = project_nodes(X_test[:tail], y_test[:tail], proj_type=UMAP)

In [ ]:
embs_tsne = project_nodes(X_test[:tail], y_test[:tail], proj_type=TSNE, n_jobs=-1)

#### Plot projections

In [ ]:
plot_nodes(embs_umap, y_test[:tail], colors, proj_type=UMAP)

In [ ]:
plot_nodes(embs_tsne, y_test[:tail], colors, proj_type=TSNE)

## Train different tree based ensemble models on node features

Implement helper functions to analyze the model's performance:
- Accuracy
- Precision
- Recall
- F1 score
- True positives
- False positives
- True negatives
- False negatives
- Confusion matrix
and plotting the confusion matrix

In [ ]:
from sklearn.base import ClassifierMixin, RegressorMixin
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, ConfusionMatrixDisplay, 
                             precision_recall_fscore_support)

In [ ]:
def plot_confusion_matrix(cmx:np.ndarray):
    """Plot confusion matrix"""
    display_labels = ['tag', 'ad']
    disp = ConfusionMatrixDisplay(confusion_matrix=cmx,
                                  display_labels=display_labels)
    disp = disp.plot(include_values=True,
                     cmap='viridis', ax=None, xticks_rotation='horizontal',
                     values_format=None, colorbar=True)
    disp.ax_.set_title('Confusion matrix')
    plt.show()

In [ ]:
def estimate_model(
        y: np.ndarray,
        y_pred: np.ndarray) -> tuple:
    """
    Validate performane on predictions
    Args:
        y (np.ndarray): true labels
        y_pred (np.ndarray): predicted values

    Returns:
        val_acc (float): validation accuracy
        pr_rc_f1 (float): validation f1 score
        rep (dict): validation report dictionary
        cmx (np.ndarray): confusion matrix
    """
    val_acc = accuracy_score(y, y_pred, sample_weight=None)
    target_names = ['tag', 'ad']

    pr_rc_f1 = precision_recall_fscore_support(y, y_pred, beta=1.0,
                                               average='binary')
    rep = classification_report(y, y_pred, target_names=target_names,
                                zero_division=0)
    cmx = confusion_matrix(y, y_pred)

    return val_acc, pr_rc_f1, rep, cmx


def validate_model(
        md: ClassifierMixin,
        X: np.ndarray, y: np.ndarray) -> tuple:
    """
    validate model performance
    Args:
        md (Union[ClassifierMixin, RegressorMixin]): model to evaluate
        X (np.ndarray): evaluation data
        y (np.ndarray): evaluation labels

    Returns:
        val_acc (float): validation accuracy
        pr_rc_f1 (float): validation F1 score
        rep (dict): validation report dictionary
        cmx: (np.ndaray): confusion matrix
    """
    y_pred = md.predict(X)
    val_acc, pr_rc_f1, rep, cmx = estimate_model(y, y_pred)

    return val_acc, pr_rc_f1, rep, cmx

In [ ]:
def validate_and_print(md, X, y):
    acc, prf1, rep, cmx = validate_model(md, X, y)
    tn, fp, fn, tp = cmx.ravel()
    pr, rc, f1_sc, _ = prf1
    print(f'accuracy: {acc:.4f}')
    print(f'precision: {pr:.4f}, recall: {rc:.4f}, f1_score: {f1_sc:.4f}')
    print(f'True positives: {tp}, false positives: {fp}, true negatives: {tn}, false negatives: {fn}')
    plot_confusion_matrix(cmx)

## About evaluation metrics

Because ads distribution is small in comparison with tags if we just create a simple model which returns  $0$  no matter of input features, it will have high accuracy.

In [ ]:
class SimpleModel(object):
    """Simple model which returns 0 no matter of input"""
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        return np.zeros(X.shape[0], dtype=int)

Now let's measure accuracy of our `SimpleModel`

In [ ]:
sm = SimpleModel()
y_pred = sm.predict(X_test)

In [ ]:
simple_acc = np.mean(y_pred == y_test)
print(f'accuracy: {simple_acc:.2f}')

## Alternative mesures

Define several estimations for binary classification:
- True positives (`TP`): model's output is $1$ while the actual label is $1$
- False positives (`FP`): model's output is $1$ while the actual label is $0$
- True negatives (`TN`): model's output is $0$ while the actual label is $0$
- False negatives (`FN`): model's output is $0$ while the actual label is $1$

Now define precision and recall:
$$
\text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}
$$
The measure of correct positive predictions overall positive predictions
<br>
and
$$
\text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}
$$
The Measure of correct positive predictions overall positive labels 

According to these measures `F1` score as:
$$
\text{F1} = 2 \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
$$
Which will estimate the model's performance on imbalanced datasets

In [ ]:
validate_and_print(sm, X_test, y_test)

As we see, besides the fact that our `SimpleModel` has high accuracy, Precision and Recall and thus `F1` score are all  $0$  which gives a more robust estimation of our model performance

## Ensemble methods

Instead of training a single model, ensemble methods use multiple models to obtain better performance
<br>
For classification majority vote technique is used and averaging for final decision for regression

Mostly weak estimators are used like `Decision Tree` with low depth

#### Bootstrap aggregating (bagging)

Bootstrap aggregating (bagging) models are generated using the same algorithm with random sub-samples of the dataset which are drawn from the original dataset randomly with bootstrap sampling method with duplication.
<br>
We will train `Random Forest` bagging classifier on our dataset

`Random Forest` <a href="https://link.springer.com/article/10.1023/A:1010933404324">paper</a> <a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html">scikit-learn</a> 

#### Boosting

Boosting incrementally builds an ensemble by training each new model instance to on the errors of the previous instance:
- `AdaBoosting` trains new models on weighted sample of training data with higher weights assigned to the part where the previous model had lower performance
- `Gradient Boosting` trains each new instance on the errors (residuals) of the previous instance
<br>
We eill train `XGBoost` (Extrime Gradient Boosting) and `LightGBM` (Light Gradient Boosting Machine) boosting classifiers on our datasets

`XGBoost` - <a href="https://arxiv.org/abs/1603.02754">paper</a> <a href="https://github.com/dmlc/xgboost">project</a> 

`LightGBM` - <a href="https://www.microsoft.com/en-us/research/publication/lightgbm-a-highly-efficient-gradient-boosting-decision-tree/">paper</a>
<a href="https://github.com/microsoft/LightGBM">project</a> 

## Train Random Forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (confusion_matrix, precision_recall_fscore_support, 
                             classification_report)

In [ ]:
## Change this parameters and train the model
n_estimators = 128
max_depth = 50
min_samples_split = 8
max_samples = 0.6

In [ ]:
random_state=0
n_jobs=workers
verbose=4

In [ ]:
rf_params = dict(n_estimators=n_estimators, max_depth=max_depth, 
                 min_samples_split=min_samples_split,
                 max_samples=max_samples)

In [ ]:
rfc = RandomForestClassifier(**rf_params, random_state=random_state, 
                             n_jobs=n_jobs, verbose=verbose)

In [ ]:
rfc = rfc.fit(X_train, y_train)

In [ ]:
print(f'model: Random Forest classifier n_estimators={n_estimators} X max_depth{max_depth}')
print()
validate_and_print(rfc, X_test, y_test)

#### Save Random Forest model

In [ ]:
from joblib import dump, load

In [ ]:
RF_MODEL_PATH = DATA_DIR / 'rf_model.joblib'
RF_MODEL_PATH

In [ ]:
dump(rfc, RF_MODEL_PATH)

#### De-serialize the model

In [ ]:
rfc_ds = load(RF_MODEL_PATH)

In [ ]:
print(f'model: Random Forest classifier n_estimators={n_estimators} X max_depth={max_depth}')
print()
validate_and_print(rfc_ds, X_test, y_test)

In [ ]:
gc.collect()

## Train GXBoost classifier

In [ ]:
import xgboost as xgb

In [ ]:
## Change this parameters and train the model
n_estimators = 512
max_depth = 8
max_samples = 0.4
learning_rate = 0.1

In [ ]:
tree_method = 'gpu_hist' #'exact' ##'approx' 
gpu_id=0 #None
random_state=0
n_jobs=workers
verbose=3

In [ ]:
xgc = xgb.XGBClassifier(n_estimators=n_estimators, use_label_encoder=False, tree_method=tree_method,
                        max_depth=max_depth, min_samples_split=min_samples_split,
                        learning_rate=learning_rate, verbosity=verbose, gpu_id=gpu_id,
                        n_jobs=n_jobs, random_state=random_state)

In [ ]:
xgc.fit(X_train, y_train, eval_set=[(X_test, y_test)], 
        eval_metric = ['error'], verbose=True)

In [ ]:
evals_result = xgc.evals_result()
evals_result

In [ ]:
print(f'model: XGBoost classifier n_estimators={n_estimators} X max_depth={max_depth}')
print()
validate_and_print(xgc, X_test, y_test)

## Serialize the model

In [ ]:
XGC_MODEL_PATH = DATA_DIR / 'xgc_model.joblib'
XGC_MODEL_PATH

In [ ]:
dump(xgc, XGC_MODEL_PATH)

## Load the model

In [ ]:
xgc_ds = load(XGC_MODEL_PATH)

In [ ]:
print(f'model: XGBoost classifier n_estimators={n_estimators} X max_depth={max_depth}')
print()
validate_and_print(xgc_ds, X_test, y_test)

In [ ]:
gc.collect()

## Train LightGBM classifierm

In [ ]:
import lightgbm as lgb

In [ ]:
## Change this parameters and train the model
n_estimators = 2048 * 2
max_depth = 7
max_samples = 0.4
learning_rate=0.1

In [ ]:
## GBM specific ##
num_leaves=min(2**max_depth, 80)
objective = 'binary'
num_leaves = 128
min_data_in_leaf = 800

######################
######################

random_state=0
n_jobs=-1
verbose=100

In [ ]:
gb_essent = dict(n_estimators=n_estimators, max_depth=max_depth, 
                 objective=objective,
                 num_leaves=num_leaves,
                 min_data_in_leaf=min_data_in_leaf,
                 learning_rate=learning_rate)

In [ ]:
gb_additional = dict(objective='binary',
                     num_leaves=num_leaves,
                     min_data_in_leaf=min_data_in_leaf,
                     is_unbalance=True,
                     boosting_type='gbdt',
                     num_boost_round=12000,
                     early_stopping_rounds=1000,
                     bagging_freq=16,
                     bagging_fraction=0.76,
                     min_gain_to_split=0.24,
                     silent=False)

In [ ]:
gb_params = {**gb_essent, **gb_additional}

In [ ]:
lgc = lgb.LGBMClassifier(**gb_params, random_state=random_state, n_jobs=n_jobs, verbose=verbose)

In [ ]:
lgc = lgc.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=4)

In [ ]:
print(f'model: LightGBM classifier n_estimators={n_estimators} X max_depth={max_depth}')
print()
validate_and_print(lgc, X_test, y_test)

## Serialize the model

In [ ]:
LGC_MODEL_PATH = DATA_DIR / 'lgc_model.joblib'
LGC_MODEL_PATH

In [ ]:
dump(lgc, LGC_MODEL_PATH)

## Load the model

In [ ]:
lgc_ds = load(LGC_MODEL_PATH)

In [ ]:
print(f'model: LightGBM classifier n_estimators={n_estimators} X max_depth={max_depth}')
print()
validate_and_print(lgc_ds, X_test, y_test)

In [ ]:
gc.collect()